In [0]:
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch
import torch.optim as optim
import time
import os
import copy
from collections import defaultdict
import matplotlib.pyplot as plt

transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
  ])

In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()
    loss_acc_history = defaultdict(list)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            loss_acc_history[phase].append((epoch_loss, epoch_acc))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, loss_acc_history

In [0]:
model = models.resnet34(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_size = 224

In [0]:
train_set = datasets.ImageFolder("data/train",transforms)
val_set = datasets.ImageFolder("data/val",transforms)

image_datasets = {'train': train_set, 'val': val_set}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], shuffle=True, num_workers=4) for x in ['train', 'val']}

In [0]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train and evaluate
model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=20)

Epoch 0/19
----------
train Loss: 1.0106 Acc: 0.5167
val Loss: 1.0946 Acc: 0.5000

Epoch 1/19
----------
train Loss: 1.2082 Acc: 0.4167
val Loss: 0.9138 Acc: 0.5417

Epoch 2/19
----------
train Loss: 0.3183 Acc: 0.8667
val Loss: 0.2723 Acc: 0.8333

Epoch 3/19
----------
train Loss: 0.0949 Acc: 1.0000
val Loss: 0.2100 Acc: 0.9167

Epoch 4/19
----------
train Loss: 0.0494 Acc: 1.0000
val Loss: 0.1314 Acc: 1.0000

Epoch 5/19
----------


In [0]:
# Plot loss curves
fig, ax = plt.subplots()
ax.set_yscale('log')
t = [x[0] for x in hist['train']]
x = [str(i) for i in range(20)]
ax.plot(x, t, label='training loss')
print('Training Losses: ', t)
t = [x[0] for x in hist['val']]
ax.plot(x, t, label='validation loss')
print('Validation Losses: ', t)

ax.set(xlabel='Epoch', ylabel='Loss',
       title='Loss v Epoch (ResNet34)')
ax.grid()
ax.legend()
plt.show()

max_train_accuracy = max([x[1] for x in hist['train']])
print('Final Train Accuracy: ', float(max_train_accuracy))

max_val_accuracy = max([x[1] for x in hist['val']])
print('Final Validation Accuracy: ', float(max_val_accuracy))